[View in Colaboratory](https://colab.research.google.com/github/kohei-shima/keras/blob/master/cifar10_cnn.ipynb)

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [0]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [78]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model = Sequential()
#model.add(Conv2D(32, (3, 3), padding='same',
#                 input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
#model.add(Conv2D(32, (3, 3)))
model.add(Conv2D(64, (3, 3)))  ##kshima
model.add(Activation('relu'))

#model.add(Conv2D(32, (3, 3))) ##kshima
#model.add(Activation('relu')) ##kshima

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(1024, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [0]:
# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

#opt = keras.optimizers.rmsprop(lr=0.001, decay=0.0) ##kshima
#opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9,beta_2=0.999,decay=0.0) ##kshima
opt = keras.optimizers.Adamax(lr=0.002, beta_1=0.9,beta_2=0.999,decay=0.0) ##kshima
#opt = keras.optimizers.Nadam(lr=0.002, beta_1=0.9,beta_2=0.999,schedule_decay=0.004) ##kshima
#opt = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False) ##kshima
#opt = keras.optimizers.Adagrad(lr=0.01,decay=0.0) ##kshima
#opt = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0) ##kshima

In [0]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [73]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        fill_mode='nearest',  # set mode for filling points outside the input boundaries
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale=None,  # set rescaling factor (applied before any other transformation)
        preprocessing_function=None,  # set function that will be applied on each input
        data_format=None,  # image data format, either "channels_first" or "channels_last" 
        validation_split=0.0)  # fraction of images reserved for validation (strictly between 0 and 1)  
        
  # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
        
  # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/10
1315/1563 [========================>.....] - ETA: 10s - loss: 1.6183 - acc: 0.4035

1563/1563 [==============================] - 68s 43ms/step - loss: 1.5751 - acc: 0.4206 - val_loss: 1.2054 - val_acc: 0.5618
Epoch 2/10
 704/1563 [============>.................] - ETA: 35s - loss: 1.2786 - acc: 0.5416

1563/1563 [==============================] - 67s 43ms/step - loss: 1.2153 - acc: 0.5666 - val_loss: 1.0088 - val_acc: 0.6387
Epoch 3/10
 464/1563 [=======>......................] - ETA: 44s - loss: 1.0736 - acc: 0.6146

1563/1563 [==============================] - 67s 43ms/step - loss: 1.0417 - acc: 0.6299 - val_loss: 0.9632 - val_acc: 0.6737
Epoch 4/10
 366/1563 [======>.......................] - ETA: 47s - loss: 0.9879 - acc: 0.6538

1563/1563 [==============================] - 59s 38ms/step - loss: 0.9378 - acc: 0.6718 - val_loss: 0.8468 - val_acc: 0.6991
Epoch 5/10
 401/1563 [======>.......................] - ETA: 38s - loss: 0.8810 - acc: 0.6924

...

1563/1563 [==============================] - 56s 36ms/step - loss: 0.8123 - acc: 0.7175 - val_loss: 0.6596 - val_acc: 0.7734
Epoch 7/10
 421/1563 [=======>......................] - ETA: 38s - loss: 0.7629 - acc: 0.7298

1563/1563 [==============================] - 55s 35ms/step - loss: 0.7624 - acc: 0.7349 - val_loss: 0.6832 - val_acc: 0.7701
Epoch 8/10
 365/1563 [======>.......................] - ETA: 39s - loss: 0.7257 - acc: 0.7467

1563/1563 [==============================] - 56s 36ms/step - loss: 0.7268 - acc: 0.7497 - val_loss: 0.6263 - val_acc: 0.7818
Epoch 9/10
 393/1563 [======>.......................] - ETA: 38s - loss: 0.7022 - acc: 0.7582

1563/1563 [==============================] - 59s 38ms/step - loss: 0.6971 - acc: 0.7588 - val_loss: 0.6162 - val_acc: 0.7904
Epoch 10/10
 417/1563 [=======>......................] - ETA: 40s - loss: 0.6740 - acc: 0.7651

1563/1563 [==============================] - 58s 37ms/step - loss: 0.6740 - acc: 0.7655 - val_loss: 0.5854 - val_acc: 0.8036


In [74]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)


Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 


In [75]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


10000/10000 [==============================] - 2s 214us/step
Test loss: 0.5854397578239441
Test accuracy: 0.8036


In [85]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: ignored